In [48]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(meta, name, bases, class_dict)
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls 

class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None
    
    def __set_name__(self, owner, name):
        print(f"self={self}, owner={owner}, name={name}")
        # called before __set__
        self.name = name
        self.internal_name = '_' + name

    def __get__(self, instance, instance_type):
        print("instance=",instance, "instance_type=",instance_type.__name__)
        print("values", instance.__dict__)
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        print(self, "instance=",instance, "value=",value)
        setattr(instance, self.internal_name, value)


In [37]:
class DatabaseRow(metaclass=Meta):
    pass 

<class '__main__.Meta'> DatabaseRow () {'__module__': '__main__', '__qualname__': 'DatabaseRow'}


In [38]:
class Customer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

cust = Customer()
print(f'{cust.first_name}, {cust.__dict__}')
cust.first_name = 'Euclid'
cust.last_name = 'Beniwal'
print(f'After {cust.first_name}, {cust.__dict__}')

<class '__main__.Meta'> Customer (<class '__main__.DatabaseRow'>,) {'__module__': '__main__', '__qualname__': 'Customer', 'first_name': <__main__.Field object at 0x7f27cf759210>, 'last_name': <__main__.Field object at 0x7f27cf75a9b0>, 'prefix': <__main__.Field object at 0x7f27cf758a00>, 'suffix': <__main__.Field object at 0x7f27cf758ca0>}
instance= <__main__.Customer object at 0x7f27cf75a890> instance_type= Customer
values {}
, {}
<__main__.Field object at 0x7f27cf759210> instance= <__main__.Customer object at 0x7f27cf75a890> value= Euclid
<__main__.Field object at 0x7f27cf75a9b0> instance= <__main__.Customer object at 0x7f27cf75a890> value= Beniwal
instance= <__main__.Customer object at 0x7f27cf75a890> instance_type= Customer
values {'_first_name': 'Euclid', '_last_name': 'Beniwal'}
After Euclid, {'_first_name': 'Euclid', '_last_name': 'Beniwal'}


In [39]:
cust.first_name

instance= <__main__.Customer object at 0x7f27cf75a890> instance_type= Customer
values {'_first_name': 'Euclid', '_last_name': 'Beniwal'}


'Euclid'

In [50]:
class BrokenCustomer:
    first_name = Field()
b = BrokenCustomer()
b.first_name = "Rahul"

self=<__main__.Field object at 0x7f27cf75a6e0>, owner=<class '__main__.BrokenCustomer'>, name=first_name
<__main__.Field object at 0x7f27cf75a6e0> instance= <__main__.BrokenCustomer object at 0x7f27e834a200> value= Rahul


In [53]:
b.__dict__

{'_first_name': 'Rahul'}

In [78]:
from functools import wraps 

def trace_func(func):
    # if hasattr(func, 'tracing'):
    #     return func 

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e 
            raise 
        finally:
            print(f'{func.__name__}({args}, {kwargs}) -> {result}')
    wrapper.tracing = True 
    return wrapper 

In [81]:
# Problem is that for logging we have to define all methods again 
# Boiler Plate Code 
class TraceDict(dict):
    @trace_func
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @trace_func
    def __setitem__(self, *args, **kwargs):
        return super().__setitem__(*args, **kwargs)
    
    @trace_func
    def __getitem__(self, *args, **kwargs):
        return super().__getitem__(*args, **kwargs)

In [86]:
import types 
trace_type = (
    types.MethodType, 
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType

)

class TraceMeta(type):
    def __new__(meta, name, bases, class_dict):
        klass = super().__new__(meta, name, bases, class_dict)

        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_type):
                wrapped = trace_func(value)
                setattr(klass, key, wrapped)
        return klass 

In [92]:
def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        if isinstance(value, trace_type):
            wrapped = trace_func(value)
            setattr(klass, key, wrapped)
    return klass

In [98]:
@trace
class SimpleTraceDict(dict):
    pass 

trace_dict = SimpleTraceDict([('HI', 1)])
trace_dict[1] = 2 
trace_dict['hi']

__format__(({}, ''), {}) -> {}
__new__((<class '__main__.SimpleTraceDict'>, [('HI', 1)]), {}) -> {}
__getitem__(({'HI': 1, 1: 2}, 'hi'), {}) -> 'hi'


KeyError: 'hi'

In [97]:
class TraceDict(dict, metaclass=TraceMeta):
    pass

In [88]:
trace_dict = TraceDict({'a': 'b'})
trace_dict['rahul'] = 'Beniwal'
trace_dict['hi']

__format__(({}, ''), {}) -> {}
__new__((<class '__main__.TraceDict'>, {'a': 'b'}), {}) -> {}
__getitem__(({'a': 'b', 'rahul': 'Beniwal'}, 'hi'), {}) -> 'hi'


KeyError: 'hi'

In [89]:
class OtherMeta(type):
    pass

class SimpleDict(dict, metaclass=OtherMeta):
    pass 

class TraceDict(SimpleDict, metaclass=TraceMeta):
    pass 

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [90]:
class OtherMeta(TraceMeta):
    pass 

class SimpleDict(dict, metaclass=OtherMeta):
    pass

class TraceDict(SimpleDict, metaclass=TraceMeta):
    pass 

__init_subclass__((), {}) -> None


In [91]:
def my_class_decorator(klass):
    klass.extra_param = 'Hello'
    return klass 

@my_class_decorator
class Myclass:
    pass

print(Myclass)
print(Myclass.extra_param)

<class '__main__.Myclass'>
Hello


In [80]:
help(TraceDict.__init__)

Help on function __init__ in module __main__:

__init__(self, *args, **kwargs)
    Initialize self.  See help(type(self)) for accurate signature.



In [76]:
trace_dict = TraceDict({'a': 'b'})
trace_dict['rahul'] = 'Beniwal'
trace_dict['hi']

__init__(({'a': 'b'}, {'a': 'b'}), {}) -> None
__setitem__(({'a': 'b', 'rahul': 'Beniwal'}, 'rahul', 'Beniwal'), {}) -> None
__getitem__(({'a': 'b', 'rahul': 'Beniwal'}, 'hi'), {}) -> 'hi'


KeyError: 'hi'

In [63]:
def function(a, b):
    c = 100
    return a,b 
function.c = 1000

In [64]:
function.__dict__

{'c': 1000}